<h1><i> Cálculo de KPI</h1></i>
<a id='Indice'></a>

## Índice

<ul>
     <LI>[1. PASOS DE PREPARACIÓN](#1.)
     <OL> 
        <ul>[1.1. Carga de las bibliotecas requeridas](#1.1.)</ul>
        <ul>[1.2. Copia de las tablas desde la validación](#1.2.)</ul>
        <ul>[1.3. Definición de las variables globales](#1.3.)</ul>
        <ul>[1.4. Carga las funciones](#1.4.)</ul>
        <ul>[1.5. Carga de datos sin procesar](#1.5.)</ul>
        <ul>[1.6. Generación de tabla de constantes con su valor más reciente](#1.6.)</ul>
        <ul>[1.7. Agregación de los datos de variables](#1.7.)</ul>
     </OL> 
</ul>
<ul>
     <LI>[2. CREACIÓN DE FUNCIONES PARA LOS KPI](#.)
     <OL> 
        <ul>[2.1. Función de cálculo del RC](#2.1.)</ul>
        <ul>[2.2. Función de cálculo del BRAE](#2.2.)</ul>
        <ul>[2.3. Función de cálculo del BRANE](#2.3.)</ul>
        <ul>[2.4. Función de cálculo del BRA](#2.4.)</ul>
        <ul>[2.5. Función de cálculo del IC_SAIDI](#2.5.)</ul>
        <ul>[2.6. Función de cálculo del IC_SAIFI](#2.6.)</ul>
        <ul>[2.7. Función de cálculo del IAAOM](#2.7.)</ul>
        <ul>[2.8. Función de cálculo del IAA](#2.8.)</ul>
        <ul>[2.9. Función de cálculo del INCD](#2.9.)</ul>
        <ul>[2.10. Función de cálculo del INCF](#2.10.)</ul>
        <ul>[2.11. Función de cálculo del IRM](#2.11.)</ul>
        <ul>[2.12. Función de cálculo del INCC](#2.12.)</ul>
        <ul>[2.13. Función de cálculo del IM](#2.13.)</ul>
        <ul>[2.14. Función de cálculo del CONP](#2.14.)</ul>
        <ul>[2.15. Función de cálculo del IA](#2.15.)</ul>
        <ul>[2.16. Función de cálculo del CD4](#2.16.)</ul>
        <ul>[2.17. Función de cálculo del CD3](#2.17.)</ul>
        <ul>[2.18. Función de cálculo del CD3-2](#2.18.)</ul>
        <ul>[2.19. Función de cálculo del CD2](#2.19.)</ul>
        <ul>[2.20. Función de cálculo del Dtcs](#2.20.)</ul>
        <ul>[2.21. Función de cálculo del CDI](#2.21.)</ul>
        <ul>[2.22. Función de cálculo del CDA](#2.22.)</ul>
        <ul>[2.23. Función de cálculo del Dt4](#2.23.)</ul>
        <ul>[2.24. Función de cálculo del Dt3 y Dt2](#2.24.)</ul>
        <ul>[2.25. Función de cálculo del Dt1](#2.25.)</ul>
     </OL> 
</ul>
<ul>
     <LI>[3. Cálculo de los KPI](#1.)
     <OL> 
        <ul>[3.1. Cálculo de los KPI](#3.1.)</ul>
     </OL> 
</ul>

<a id='1.'></a>

## 1. Pasos de preparación   [Volver al inicio](#Indice)

<a id='1.1.'></a>

### 1.1. Carga de las bibliotecas requeridas   [Volver al inicio](#Indice)

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML
from dateutil.rrule import rrule, MONTHLY

<a id='1.2.'></a>

### 1.2. Copia de las tablas desde la validación   [Volver al inicio](#Indice)

In [2]:
!cp ~/DaVinciv14/tablas/ex_constantes.txt ~/jupyter_notebook/files/ex_constantes.txt
!cp ~/DaVinciv14/tablas/ex_variables.txt ~/jupyter_notebook/files/ex_variables.txt

<a id='1.3.'></a>

### 1.3. Definición de las variables globales   [Volver al inicio](#Indice)

In [3]:
global datos_cons
global datos_var
global datos_var_anual
global descripcion
global timestamp
global timestamp_id
global tr_niveles
global log_file
diffHours = 7
timestamp = ( datetime.today() - timedelta(hours = diffHours, minutes=0) ).strftime('%Y-%m-%d %H:%M:%S')
timestamp_id = int(''.join(filter(str.isdigit, timestamp)))

<a id='1.4.'></a>

### 1.4. Carga de funciones  [Volver al inicio](#Indice)

In [4]:
''' Función que retorna el último valor de una constante '''
def get_cons_last_value(cons):
    return float(datos_cons_raw[datos_cons_raw['CONSTANTE']==cons].sort_values(by='TIMESTAMP', ascending=False)['VALOR'].head(1).values)

''' Función que retorna el nombre asociado al codigo '''
def get_var_name(code):
    return descripcion[descripcion['CODIGO']==code].index.values[0]

''' Función que retorna las fechas entre 14 y 3 meses atrás '''
def dates_to_iter(t,m):
    d = datetime.strptime(str(t)+"-"+str(m)+"-01", "%Y-%m-%d")
    start = d - relativedelta(months=14)
    end = d - relativedelta(months=3)
    return [dt for dt in rrule(MONTHLY, dtstart=start, until=end)]

''' Función que guarda localmente los valores intermedios '''
def save_to_local_file(var_cons_name,anio, periodo, value, level):
    global timestamp
    global tr_niveles
    tr_niveles = tr_niveles.append(pd.DataFrame([[var_cons_name,anio,periodo,value,level,timestamp]],columns=tr_niveles.columns.values))

''' Función que mueve los archivos locales al hdfs '''
def move_local_to_hdfs(source_path, destination_path):
    os.system('hdfs dfs -copyFromLocal -f ' + source_path + ' ' + destination_path)

''' Función que imprime y guarda el log de ejecución '''
def log(name, status):
    global log_file
    global timestamp
    global timestamp_id
    if status==0:
        log_msg = 'Info: KPI '+ name +' - calculado correctamente.'
    elif status==1:
        log_msg = 'Error: Falta el dato '+ str(name.split('$')[1]) +' para calcular '+str(name.split('$')[0])+'.'
    elif status==2:
        log_msg = 'Error: División por cero al calcular el KPI '+ name +'.'
    print(log_msg)
    timestamp_now = ( datetime.today() - timedelta(hours = diffHours, minutes=0) ).strftime('%Y-%m-%d %H:%M:%S')
    log_file = log_file.append(pd.DataFrame([[timestamp_id,timestamp,timestamp_now,log_msg]],columns=['ID','TIMESTAMP GLOBAL','TIMESTAMP CELDA','MENSAJE']))

''' Función que retorna el dato de las variables '''
def get_var(name,t,m):
    global datos_var
    global descripcion
    if ('IReactivaj' not in name) and ('DTCj,R' not in name):
        return float(datos_var[descripcion[descripcion['CODIGO']==name].index.values].loc[t,m])
    elif 'DTCj,R' in name:
        return float(datos_var['DEMANDA_TOT_COMERC_MERC_COMERC_OR_STR_R_MES_PERD'].loc[t,m])
    else:
        return float(sum(datos_var[descripcion[descripcion['CODIGO']==name].index.values].loc[t,m]))

''' Función que retorna el dato de las variables anualizadas '''
def get_var_annualized(name,t):
    global datos_var_anual
    global descripcion
    if ('IReactivaj' not in name) and ('DTCj,R' not in name):
        return float(datos_var_anual[descripcion[descripcion['CODIGO']==name].index.values].loc[t])
    elif 'DTCj,R' in name:
        return float(datos_var_anual['DEMANDA_TOT_COMERC_MERC_COMERC_OR_STR_R_MES_PERD'].loc[t])
    else:
        return float(sum(datos_var_anual[descripcion[descripcion['CODIGO']==name].index.values].loc[t]))

''' Función que retorna el dato de las constantes '''
def get_constant(name):
    global datos_cons
    global descripcion
    if ('IReactivaj' not in name) :
        return float(datos_cons['VALOR'].loc[descripcion[descripcion['CODIGO']==name].index.values])
    else:
        return float(sum(datos_cons['VALOR'].loc[descripcion[descripcion['CODIGO']==name].index.values]))

'''Función que retorna el mes y año anterior cuando se selecciona enero'''
def month_correction(year, month):
    if month==0:
        return [year-1, 12]
    else:
        return [year, month]

<a id='1.5.'></a>

### 1.5. Carga de datos sin procesar   [Volver al inicio](#Indice)

In [5]:
datos_var          = pd.read_table('files/ex_variables.txt', sep='$', index_col=[0,1])
datos_cons_raw     = pd.read_table('files/ex_constantes.txt', sep='$')
descripcion        = pd.read_table('files/val_descripcion.txt',sep='$', index_col=[1])
log_file           = pd.DataFrame(columns=['ID','TIMESTAMP GLOBAL','TIMESTAMP CELDA','MENSAJE'])
tr_niveles         = pd.DataFrame(columns=['INDICADOR','ANIO','PERIODO','VALOR','NIVEL','TIMESTAMP'])

<a id='1.6.'></a>

### 1.6. Generación de tabla de constantes con su valor más reciente   [Volver al inicio](#Indice)

<a id='bottom'></a>

In [6]:
datos_cons_raw['ANIO'] = datos_cons_raw['TIMESTAMP'].apply(lambda x: str(x)[:4])
datos_cons_raw['PERIODO'] = datos_cons_raw['TIMESTAMP'].apply(lambda x: str(x)[5:7])
datos_cons_raw = datos_cons_raw.set_index(['ANIO','PERIODO'])
rows = list(datos_cons_raw['CONSTANTE'].unique())
datos_cons = pd.DataFrame(columns = ['VALOR'] + ['CONSTANTE'])
datos_cons = datos_cons.set_index(['CONSTANTE'])
for i in rows:
    datos_cons.loc[(i),'VALOR'] = get_cons_last_value(i)

<a id='1.7.'></a>

### 1.7. Agregación de los datos de variables   [Volver al inicio](#Indice)

In [7]:
datos_var_anual = pd.DataFrame(columns=datos_var.columns,index=datos_var.index.get_level_values('ANIO').unique())
for i in datos_var.columns.values:
    for j in datos_var.index.get_level_values('ANIO').unique().values:
        datos_temp = datos_var[i].loc[j,([k for k in range(0,13)])]
        datos_var_anual[i].loc[j] = sum(datos_temp) if sum(datos_temp.isnull())==len(datos_temp) else datos_temp.sum(skipna=True)

In [8]:
datos_anualizados = datos_var_anual.T.unstack('ANIO').reset_index()
datos_anualizados.columns=['ANIO','VARIABLE','VALOR']
datos_anualizados = datos_anualizados[pd.isna(datos_anualizados['VALOR'])==False]
datos_anualizados['ID'] = timestamp_id
datos_anualizados['TIMESTAMP']= timestamp
datos_anualizados = datos_anualizados[['ID','TIMESTAMP','VARIABLE','ANIO','VALOR']]
datos_anualizados = datos_anualizados.reset_index(drop=True)

<a id='2.'></a>

## 2. Creación de funciones para los KPI   [Volver al inicio](#Indice)

<a id='2.1.'></a>

### 2.1. Función de cálculo del RC    [Volver al inicio](#Indice)
<hr>
$\Large RC_\color{blue}{j,n,t}=RCBIA_\color{blue}{j,n,t}+RCNA_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large RC_\color{blue}{j,n,t}=RCBIA_\color{blue}{j,n,t-1}+RCNA_\color{blue}{j,n,t-1}$

In [9]:
''' KPI Anual'''
def RCjnt(n,t):
    name ='RCj,'+str(n)+','+str(t)+':'
    in_progress=''
    t=t-1
    try:
        ''' RCBIA Variable Mensual '''
        in_progress='$RCBIA'
        if t>=2019:
            value = get_var_annualized('RCBIAj,'+str(n)+',t:',t)
            RCBIA = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('RCBIAj,'+str(n)+',t:_2018')
            RCBIA = value if pd.isnull(value)==False else 1/0

        ''' RCNA Variable Mensual '''
        in_progress='$RCNA'
        if t>=2019:
            value = get_var_annualized('RCNAj,'+str(n)+',t:',t)
            RCNA = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('RCNAj,'+str(n)+',t:_2018')
            RCNA = value if pd.isnull(value)==False else 1/0

        output = RCBIA + RCNA
        log(name,0)
        save_to_local_file(name,t,0,output,8)
        return output
    except:
        log(name+in_progress,1)

<a id='2.2.'></a>

<a id='2.2.'></a>

### 2.2. Función de cálculo del BRAE    [Volver al inicio](#Indice)
<hr>
$\Large BRAE_\color{blue}{j,n,t}=BRAE_\color{blue}{j,n,t-1}-RC_\color{blue}{j,n,t}+BRAEN_\color{blue}{j,n,t}-BRAFO_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large BRAE_\color{blue}{j,n,t}=BRAE_\color{blue}{j,n,t-1}-RC_\color{blue}{j,n,t}+BRAEN_\color{blue}{j,n,t-1}-BRAFO_\color{blue}{j,n,t-1}$

In [10]:
''' KPI Anual '''
def BRAEjnt(n,t):
    name ='BRAEj,'+str(n)+','+str(t)+':'
    in_progress=''
    t=t-1
    try:
        ''' BRAE KPI Anual, el 2018 es constante '''
        in_progress='$BRAEt_1'
        if t >= 2019:
            value = BRAEjnt(n,t)
            BRAEt_1 = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('BRAEj,'+str(n)+',0')
            BRAEt_1 = value if pd.isnull(value)==False else 1/0

        ''' RC KPI Anual '''
        in_progress='$RC'
        value = RCjnt(n,t+1)
        RC = value if pd.isnull(value)==False else 1/0
        
        ''' BRAEN Variable Mensual '''
        in_progress='$BRAEN'
        if t >= 2019:
            value = get_var_annualized('BRAENj,'+str(n)+',t',t)
            BRAEN = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('BRAENj,'+str(n)+',t_2018')
            BRAEN = value if pd.isnull(value)==False else 1/0
    
        ''' BRAFO Variable Mensual '''
        in_progress='$BRAFO'
        if t >= 2019:
            value = get_var_annualized('BRAFOj,'+str(n)+',t',t)
            BRAFO = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('BRAFOj,'+str(n)+',t_2018')
            BRAFO = value if pd.isnull(value)==False else 1/0

        output = BRAEt_1 - RC + BRAEN - BRAFO
        log(name,0)
        save_to_local_file(name,t,0,output,7)
        return output
    except:
        log(name+in_progress,1)

<a id='2.3.'></a>

### 2.3. Función de cálculo del BRANE    [Volver al inicio](#Indice)
<hr>
$\Large BRANE_\color{blue}{j,n,t}=NE \cdot BRAE_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [11]:
''' KPI Anual '''
def BRANEjnt(n,t):
    name ='BRANEj,'+str(n)+','+str(t)+''
    in_progress=''
    t=t-1
    try:
        ''' NE Constante '''
        in_progress='$NE'
        value = get_constant('NE')
        NE = value if pd.isnull(value)==False else 1/0

        ''' BRAE KPI Anual '''
        in_progress='$BRAE'
        value = BRAEjnt(n,t+1)
        BRAE = value if pd.isnull(value)==False else 1/0

        output = NE*BRAE
        log(name,0)
        save_to_local_file(name,t,0,output,6)
        return output
    except:
        log(name+in_progress,1)

<a id='2.4.'></a>

### 2.4. Función de cálculo del BRA    [Volver al inicio](#Indice)
<hr>
$\Large BRA_\color{blue}{j,n,t}= BRAE_\color{blue}{j,n,t}+BRANE_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [12]:
''' KPI Anual '''
def BRAjnt(n,t):
    name = 'BRAj,'+str(n)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' BRAE KPI Anual '''
        in_progress='$BRAE'
        value = BRAEjnt(n,t+1)
        BRAE = value if pd.isnull(value)==False else 1/0
        
        ''' BRANE KPI Anual '''
        in_progress='$BRANE'
        value = BRANEjnt(n,t+1)
        BRANE = value if pd.isnull(value)==False else 1/0
        
        output = BRAE + BRANE
        log(name,0)
        save_to_local_file(name,t,0,output,5)
        return output
    except:
        log(name+in_progress,1)

<a id='2.5.'></a>

### 2.5. Función de cálculo del IC_SAIDI    [Volver al inicio](#Indice)
<hr>
$\Large IC\_SAIDI_\color{blue}{t,j}= If\_SAIDI_\color{blue}{j,t}+Iv\_SAIDI_\color{blue}{j,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large IC\_SAIDI_\color{blue}{t,j}= If\_SAIDI_\color{blue}{j,t-1}+Iv\_SAIDI_\color{blue}{j,t-1}$

In [13]:
''' KPI Anual '''
def IC_SAIDItj(t):
    name = 'IC_SAIDI'+str(t)+',j'
    in_progress=''
    t=t-1
    try:
        ''' If_SAIDI Variable Anual '''
        in_progress='$If_SAIDI'
        value = get_var('If_SAIDIj,t',t,0)
        If_SAIDI = value if pd.isnull(value)==False else 1/0
        
        ''' Iv_SAIDI Variable Anual '''
        in_progress='$Iv_SAIDI'
        value = get_var('Iv_SAIDIj,t',t,0)
        Iv_SAIDI = value if pd.isnull(value)==False else 1/0

        output = If_SAIDI + Iv_SAIDI
        log(name,0)
        save_to_local_file(name,t,0,output,5)
        return output
    except:
        log(name+in_progress,1)

<a id='2.6.'></a>

### 2.6. Función de cálculo del IC_SAIFI    [Volver al inicio](#Indice)
<hr>
$\Large IC\_SAIFI_\color{blue}{t,j}= If\_SAIFI_\color{blue}{j,t}+Iv\_SAIFI_\color{blue}{j,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large IC\_SAIFI_\color{blue}{t,j}= If\_SAIFI_\color{blue}{j,t-1}+Iv\_SAIFI_\color{blue}{j,t-1}$

In [14]:
''' KPI Anual '''
def IC_SAIFItj(t):
    name = 'IC_SAIFI'+str(t)+',j'
    in_progress=''
    t=t-1
    try:
        ''' If_SAIFI Variable Anual '''
        in_progress='$If_SAIFI'
        value = get_var('If_SAIFIj,t',t,0)
        If_SAIFI = value if pd.isnull(value)==False else 1/0

        ''' Iv_SAIFI Variable Anual '''
        in_progress='$Iv_SAIFI'
        value = get_var('Iv_SAIFIj,t',t,0)
        Iv_SAIFI = value if pd.isnull(value)==False else 1/0

        output = If_SAIFI + Iv_SAIFI
        log(name,0)
        save_to_local_file(name,t,0,output,5)
        return output
    except:
        log(name+in_progress,1)

### 2.7. Función de cálculo del IAAOM    [Volver al inicio](#Indice)
<hr>
$\Large IAAOM_\color{blue}{j,n,t}= AOMbase_\color{blue}{j,n}+AOMNI_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large IAAOM_\color{blue}{j,n,t}= AOMbase_\color{blue}{j,n}+AOMNI_\color{blue}{j,n,t-1}$

In [15]:
''' KPI Anual '''
def IAAOMjnt(n,t):
    name = 'IAAOMj,'+str(n)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' AOMbase Constante '''
        in_progress='$AOMbase'
        value = get_constant('AOMbasej,'+str(n))
        AOMbase = value if pd.isnull(value)==False else 1/0
        
        ''' AOMNI Variable Mensual '''
        in_progress='$AOMNI'
        '''Modificado para la validación'''
        if t==2018:
            AOMNI=0
        else:
            value = get_var_annualized('AOMNIj,'+str(n)+',t',t)
            AOMNI = value if pd.isnull(value)==False else 1/0
        
        output = AOMbase + AOMNI
        log(name,0)
        save_to_local_file(name,t,0,output,4)
        return output
    except:
        log(name+in_progress,1)

<a id='2.8.'></a>

### 2.8. Función de cálculo del IAA    [Volver al inicio](#Indice)
<hr>
$\Large IAA_\color{blue}{j,n,t}= BRA_\color{blue}{j,n,t}\cdot r+RC_\color{blue}{j,n,t}+BRT_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large IAA_\color{blue}{j,n,t}= BRA_\color{blue}{j,n,t}\cdot r+RC_\color{blue}{j,n,t}+BRT_\color{blue}{j,n,t}$

In [16]:
''' KPI Anual '''
def IAAjnt(n,t):
    name = 'IAAj,'+str(n)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' BRA KPI Anual '''
        in_progress='$BRA'
        value = BRAjnt(n,t+1)
        BRA = value if pd.isnull(value)==False else 1/0
        
        ''' RC KPI Anual '''
        in_progress='$RC'
        value = RCjnt(n,t+1)
        RC = value if pd.isnull(value)==False else 1/0
        
        ''' BRT Variable Anual '''
        in_progress='$BRT'
        value = get_var('BRTj,'+str(n)+',t',t+1,0)
        BRT = value if pd.isnull(value)==False else 1/0
        
        ''' r Constante '''
        in_progress='$r'
        value = get_constant('r: ')
        r = value if pd.isnull(value)==False else 1/0
        
        output = BRA*r + RC + BRT
        log(name,0)
        save_to_local_file(name,t,0,output,4)
        return output
    except:
        log(name+in_progress,1)

<a id='2.9.'></a>

### 2.9. Función de cálculo del INCD    [Volver al inicio](#Indice)
<hr>
$\Large INCD_\color{blue}{j,n,t}= IC\_SAIDI_\color{blue}{t,j}\cdot \dfrac{BRAE_\color{blue}{j,n,0}}{\sum\limits_{n=1}^{3}BRAE_\color{blue}{j,n,0}}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [17]:
''' KPI Anual '''
def INCDjnt(n,t):
    name = 'INCDj,'+str(n)+','+str(t)
    in_progress=''
    t=t-1
    try:
        '''IC_SAIDI KPI Anual'''
        in_progress = '$IC_SAIDI'
        value = IC_SAIDItj(t+1)
        IC_SAIDI = value if pd.isnull(value)==False else 1/0
        
        '''BRAEjn0 Constante'''
        in_progress = '$BRAE'
        value = get_constant('BRAEj,'+str(n)+',0')
        BRAE = value if pd.isnull(value)==False else 1/0
        
        '''SUM_BRAEjn0 Constante sumada para 3 niveles de tensión'''
        in_progress = '$SUM_BRAE'
        value = sum(get_constant('BRAEj,'+str(i)+',0') for i in [1,2,3])
        SUM_BRAE = value if pd.isnull(value)==False else 1/0
        
        try:
            if SUM_BRAE==0:
                1/0
            else:
                output = IC_SAIDI*(BRAE/SUM_BRAE)
                log(name,0)
                save_to_local_file(name,t,0,output,4)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.10.'></a>

### 2.10. Función de cálculo del INCF    [Volver al inicio](#Indice)
<hr>
$\Large INCF_\color{blue}{j,n,t}= IC\_SAIFI_\color{blue}{t,j}\cdot \dfrac{BRAE_\color{blue}{j,n,0}}{\sum\limits_{n=1}^{3}BRAE_\color{blue}{j,n,0}}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [18]:
''' KPI Anual '''
def INCFjnt(n,t):
    name = 'INCFj,'+str(n)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' IC_SAIFI KPI Anual '''
        in_progress='$IC_SAIFI'
        value = IC_SAIFItj(t+1)
        IC_SAIFI = value if pd.isnull(value)==False else 1/0
        
        ''' BRAEjn0 Constante '''
        in_progress='$BRAE'
        value = get_constant('BRAEj,'+str(n)+',0')
        BRAE = value if pd.isnull(value)==False else 1/0
        
        ''' SUM_BRAEjn0 Constante sumada para 3 niveles de tensión '''
        in_progress='$SUM_BRAE'
        value = sum(get_constant('BRAEj,'+str(i)+',0') for i in [1,2,3])
        SUM_BRAE = value if pd.isnull(value)==False else 1/0
        
        try:
            if SUM_BRAE==0:
                1/0
            else:
                output = IC_SAIFI*(BRAE/SUM_BRAE)
                log(name,0)
                save_to_local_file(name,t,0,output,4)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.11.'></a>

### 2.11. Función de cálculo del IRM    [Volver al inicio](#Indice)
<hr>
Para $n={1,2,3}$

$\Large IRM_\color{blue}{j,n,t}= IRespaldo_\color{blue}{j,n,t}+ IMunts_\color{blue}{j,n,t} + IReactiva_\color{blue}{j,n,t}$

Para $n=4$

$\Large IRM_\color{blue}{j,n,t}= IRespaldo_\color{blue}{j,n,t}+ IReactiva_\color{blue}{j,n,t}$

<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>

Para $n={1,2,3}$

$\Large IRM_\color{blue}{j,n,t}= IRespaldo_\color{blue}{j,n,t-1}+ IMunts_\color{blue}{j,n,t-1} + IReactiva_\color{blue}{j,n,t-1}$

Para $n=4$

$\Large IRM_\color{blue}{j,n,t}= IRespaldo_\color{blue}{j,n,t-1}+ IReactiva_\color{blue}{j,n,t-1}$

In [19]:
''' KPI Anual '''
def IRMjnt(n,t):
    name = 'IRMj,'+str(n)+','+str(t)+':'
    in_progress=''
    t=t-1
    try:
        ''' IRespaldo Variable Mensual '''
        in_progress='$IRespaldo'
        if t>=2019:
            value = get_var_annualized('IRespaldoj,'+str(n)+',t',t)
            IRespaldo = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('IRespaldoj,'+str(n)+',t_2018')
            IRespaldo = value if pd.isnull(value)==False else 1/0
        
        ''' IReactiva Variable Mensual '''
        in_progress='$IReactiva'
        if t>=2019:
            value = get_var_annualized('IReactivaj,'+str(n)+',t:',t)
            IReactiva = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('IReactivaj,'+str(n)+',t:_2018')
            IReactiva = value if pd.isnull(value)==False else 1/0
        
        if t>=2019:
            if n!=4:
                ''' IMunts Variable Mensual '''
                in_progress='$IMunts'
                value = get_var_annualized('IMuntsj,'+str(n)+',t:',t)
                IMunts = value if pd.isnull(value)==False else 1/0
                output = IRespaldo + IMunts + IReactiva
                log(name,0)
                save_to_local_file(name,t,0,output,4)
                return output
            else:
                output = IRespaldo + IReactiva
                log(name,0)
                save_to_local_file(name,t,0,output,4)
                return output
        else:
            if n!=4:
                ''' IMunts Variable Mensual '''
                in_progress='$IMunts'
                value = get_constant('IMuntsj,'+str(n)+',t:_2018')
                IMunts = value if pd.isnull(value)==False else 1/0
                output = IRespaldo + IMunts + IReactiva
                log(name,0)
                save_to_local_file(name,t,0,output,4)
                return output
            else:
                output = IRespaldo + IReactiva
                log(name,0)
                save_to_local_file(name,t,0,output,4)
                return output
    except:
        log(name+in_progress,1)

<a id='2.12.'></a> 

### 2.12. Función de cálculo del INCC    [Volver al inicio](#Indice)

<hr>

$\Large INCC_\color{blue}{j,n,t}= INCD_\color{blue}{j,n,t}+ INCF_\color{blue}{j,n,t}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [20]:
''' KPI Anual '''
def INCCjnt(n,t):
    name = 'INCCj,'+str(n)+','+str(t)+':'
    in_progress=''
    t=t-1
    try:
        ''' INCD KPI Anual '''
        in_progress='$INCD'
        value = INCDjnt(n,t+1)
        INCD = value if pd.isnull(value)==False else 1/0
        
        ''' INCF KPI Anual '''
        in_progress='$INCF'
        value = INCFjnt(n,t+1)
        INCF = value if pd.isnull(value)==False else 1/0
        
        output = INCD + INCF
        log(name,0)
        save_to_local_file(name,t,0,output,3)
        return output
    except:
        log(name+in_progress,1)

<a id='2.13.'></a>

### 2.13. Función de cálculo del IM    [Volver al inicio](#Indice)
<hr>
$\Large IM_\color{blue}{j,4,R,t}= \bigg[IAA_\color{blue}{j,4,t} \cdot fM + \dfrac{IAAOM_\color{blue}{j,4,t}-IRM_\color{blue}{j,4,t}}{12} \bigg] \cdot \dfrac{IPP_\color{blue}{m-1}}{IPP_\color{blue}{0}} + \sum\limits_{c=1}^{c_\color{blue}{j,R}}{IE_\color{blue}{j,c,R,m}}-CSTR_\color{blue}{j,R,m-1}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$R$ no afecta en el cálculo

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

In [21]:
''' KPI Mensual '''
def IMj4Rmt(m,t):
    n=4
    name = 'IMj,'+str(n)+'R,'+str(m)+','+str(t)+':'
    in_progress=''
    t=t-1
    try:
        ''' IAA KPI Anual '''
        in_progress='$IAA'
        value = IAAjnt(n,t+1)
        IAA = value if pd.isnull(value)==False else 1/0
        
        ''' fM Constante '''
        in_progress='$fM'
        value = get_constant('fM: ')
        fM = value if pd.isnull(value)==False else 1/0
        
        ''' IAAOM KPI Anual '''
        in_progress='$IAAOM'
        value = IAAOMjnt(n,t+1)
        IAAOM = value if pd.isnull(value)==False else 1/0
        
        ''' IRM KPI Anual '''
        in_progress='$IRM'
        value = IRMjnt(n,t+1)
        IRM = value if pd.isnull(value)==False else 1/0
        
        ''' IPP Variable Mensual '''
        in_progress='$IPP'
        value = get_var('IPPm-1:',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        IPP = value if pd.isnull(value)==False else 1/0
        
        ''' IPP0 Constante '''
        in_progress='$IPP0'
        value = get_constant('IPPo:')
        IPP0 = value if pd.isnull(value)==False else 1/0
        
        ''' IE Variable Mensual '''
        in_progress='$SUM_IE'
        value = get_var('IEj,c,R,m',month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        SUM_IE =  value if pd.isnull(value)==False else 1/0
        
        ''' CSTR Variable Mensual '''
        in_progress='$CSTR'
        value = get_var('CSTRj,R,m-1',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        CSTR = value if pd.isnull(value)==False else 1/0
        
        try:
            if IPP0==0:
                1/0
            else:
                output = (IAA*fM + (IAAOM-IRM)/12)*(IPP/IPP0)+ SUM_IE - CSTR
                log(name,0)
                save_to_local_file(name,t,m,output,3)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.14.'></a>

### 2.14. Función de cálculo del CONP    [Volver al inicio](#Indice)
<hr>
$\Large CONP_\color{blue}{j,n,t}= CONP_\color{blue}{j,t}\cdot \dfrac{BRAE_\color{blue}{j,n,0}}{\sum\limits_{n=1}^{3}BRAE_\color{blue}{j,n,0}}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large CONP_\color{blue}{j,n,t}= CONP_\color{blue}{j,t-1}\cdot \dfrac{BRAE_\color{blue}{j,n,0}}{\sum\limits_{n=1}^{3}BRAE_\color{blue}{j,n,0}}$

In [22]:
''' KPI Anual '''
def CONPjnt(n,t):
    name = 'CONPj,'+str(n)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' CONP Variable Mensual '''
        in_progress='$CONP'        
        if t>=2019:
            value = get_var_annualized('CONPj,t',t)
            CONP = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('CONPj,t_2018')
            CONP = value if pd.isnull(value)==False else 1/0
        
        ''' Constante Variable Anual '''
        in_progress='$BRAE'
        value = get_constant('BRAEj,'+str(n)+',0')
        BRAE = value if pd.isnull(value)==False else 1/0
        
        ''' SUM_BRAE Constante sumada para 3 niveles de tensión '''
        in_progress='$SUM_BRAE'
        value = sum(get_constant('BRAEj,'+str(i)+',0') for i in [1,2,3])
        SUM_BRAE = value if pd.isnull(value)==False else 1/0
        
        try:
            if SUM_BRAE==0:
                1/0
            else:
                output = CONP*(BRAE/SUM_BRAE)
                log(name,0)
                save_to_local_file(name,t,0,output,3)
                return output
        except:
            log(name,2)
    except:
        log(name+SUM_BRAE,1)

<a id='2.15.'></a>

### 2.15. Función de cálculo del IA    [Volver al inicio](#Indice)
<hr>
$\Large IA_\color{blue}{j,n,m,t}= \bigg[IAA_\color{blue}{j,n,t} \cdot fM + \dfrac{IAAOM_\color{blue}{j,n,t}-IRM_\color{blue}{j,n,t}}{12} \bigg] \cdot \dfrac{IPP_\color{blue}{m-1}}{IPP_\color{blue}{0}}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 2,3\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

In [23]:
''' KPI Mensual '''
def IAjnmt(n,m,t):
    name = 'IAj,'+str(n)+','+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' IAA KPI Anual '''
        in_progress='$IAA'
        value = IAAjnt(n,t+1)
        IAA = value if pd.isnull(value)==False else 1/0
        
        ''' fM Constante '''
        in_progress='$fM'
        value = get_constant('fM: ')
        fM = value if pd.isnull(value)==False else 1/0
        
        ''' IAAOM KPI Anual '''
        in_progress='$IAAOM'
        value = IAAOMjnt(n,t+1)
        IAAOM = value if pd.isnull(value)==False else 1/0
        
        ''' IRM KPI Anual '''
        in_progress='$IRM'
        value = IRMjnt(n,t+1)
        IRM = value if pd.isnull(value)==False else 1/0
        
        ''' IPP Variable mensual '''
        in_progress='$IPP'
        value = get_var('IPPm-1:',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        IPP = value if pd.isnull(value)==False else 1/0
        
        ''' IPP0 Constante '''
        in_progress='$IPP0'
        value = get_constant('IPPo:')
        IPP0 = value if pd.isnull(value)==False else 1/0
        
        try:
            if IPP0==0:
                1/0
            else:
                output = (IAA*fM + (IAAOM-IRM)/12)*(IPP/IPP0)
                log(name,0)
                save_to_local_file(name,t,m,output,3)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

$\Large IA_\color{blue}{j,1,m,t}= \bigg[IAA_\color{blue}{j,1,t} \cdot fM - \dfrac{IRM_\color{blue}{j,1,t}+OI_\color{blue}{j,1,t-1}}{12} \bigg] \cdot \dfrac{IPP_\color{blue}{m-1}}{IPP_\color{blue}{0}}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large IA_\color{blue}{j,1,m,t}= \bigg[IAA_\color{blue}{j,1,t} \cdot fM - \dfrac{IRM_\color{blue}{j,1,t}+OI_\color{blue}{j,1,t-1}}{12} \bigg] \cdot \dfrac{IPP_\color{blue}{m-1}}{IPP_\color{blue}{0}}$

In [24]:
''' KPI Mensual '''
def IAj1mt(m,t):
    n = 1
    name = 'IAj,'+str(n)+','+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' IAA KPI Anual '''
        in_progress='$IAA'
        value = IAAjnt(n,t+1)
        IAA = value if pd.isnull(value)==False else 1/0
        
        ''' fM Constante '''
        in_progress='$fM'
        value = get_constant('fM: ')
        fM = value if pd.isnull(value)==False else 1/0
        
        ''' OI Variable Mensual, 2018 es constante '''
        in_progress='$OI'
        if t>=2019:
            value = get_var_annualized('OIj,1,t-1',t)
            OI = value if pd.isnull(value)==False else 1/0
        else:
            value = get_constant('OIj,1,t-1_2018')
            OI = value if pd.isnull(value)==False else 1/0
        
        ''' IRM KPI Anual '''
        in_progress='$IRM'
        value = IRMjnt(n,t+1)
        IRM = value if pd.isnull(value)==False else 1/0
        
        ''' IPP Variable mensual '''
        in_progress='$IPP'
        value = get_var('IPPm-1:',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        IPP = value if pd.isnull(value)==False else 1/0
        
        ''' IPP0 Constante '''
        in_progress='$IPP0'
        value = get_constant('IPPo:')
        IPP0 = value if pd.isnull(value)==False else 1/0
        
        try:
            if IPP0==0:
                1/0
            else:
                output = (IAA*fM - (IRM + OI)/12)*(IPP/IPP0)
                log(name,0)
                save_to_local_file(name,t,m,output,3)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.16.'></a>

### 2.16. Función de cálculo del CD4    [Volver al inicio](#Indice)
<hr>
$\Large CD_\color{blue}{4,R,m,t}=\dfrac{\sum\limits_{j=1}^{J}IM_\color{blue}{j,4,R,m,t}+\sum\limits_{r=1}^{R}IMTR_\color{blue}{r,R,m,t}}{\sum\limits_{j=1}^{JR}DTC_\color{blue}{j,R,m-1}}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

In [25]:
''' KPI Mensual '''
''' Verificar el area que se debe considerar '''
def CD4Rmt(m,t):
    name = 'CD4,R,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        n=4
        ''' SUM_IM KPI Mensual '''
        in_progress='$SUM_IM'
        value = IMj4Rmt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        SUM_IM = value if pd.isnull(value)==False else 1/0
        
        ''' SUM_IMRT Variable Mensual '''
        in_progress='$SUM_IMTR'
        value = get_var('IMTRr,R,m,t',month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        SUM_IMTR = value if pd.isnull(value)==False else 1/0
        
        ''' SUM_DTC Variable Mensual '''
        in_progress='$SUM_DTC'
        value = get_var('DTCj,R,m-1',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        SUM_DTC = value if pd.isnull(value)==False else 1/0
        
        try:
            if SUM_DTC==0:
                1/0
            else:
                output = (SUM_IM + SUM_IMTR)/SUM_DTC
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.17.'></a>

### 2.17. Función de cálculo del CD3    [Volver al inicio](#Indice)
<hr>
$\Large CD_\color{blue}{3,j,m,t}=\dfrac{IA_\color{blue}{j,3,m,t}+\dfrac{1}{12}\cdot O_\color{blue}{j,3}}{\bigg (1-P_\color{blue}{j,3,t}\bigg )\cdot \dfrac{1}{12} \cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,3,ma}}}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large CD_\color{blue}{3,j,m,t}=\dfrac{IA_\color{blue}{j,3,m,t}+\dfrac{1}{12}\cdot O_\color{blue}{j,3}}{\bigg (1-P_\color{blue}{j,3,t-1}\bigg )\cdot \dfrac{1}{12} \cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,3,ma}}}$

In [26]:
''' KPI Mensual '''
def CDn(n,m,t):
    name = 'CD'+str(n)+',j,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' IA KPI mensual '''
        in_progress='$IA'
        if n in [2,3]:
            value = IAjnmt(n,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            IA = value if pd.isnull(value)==False else 1/0
        elif n ==1:
            value = IAj1mt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            IA = value if pd.isnull(value)==False else 1/0
        
        ''' O Constante '''
        in_progress='$O'
        value = get_constant('Oj,'+str(n))
        Ojn = value if pd.isnull(value)==False else 1/0
        
        ''' P Variable Anual '''
        in_progress='$P'
        if t>=2019:
            value = get_var_annualized('Pj,'+str(n)+',t',t)
            Pjnt = value if pd.isnull(value)==False else 1/0
        else:
            value = get_var('Pj,'+str(n)+',t',t,0)
            Pjnt = value if pd.isnull(value)==False else 1/0

        ''' Ee Variable Mensual '''
        in_progress='$sumEe'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('Eej,'+str(n)+',ma',i.year,i.month))
        value = sum(accum)
        sumEejnma = value if pd.isnull(value)==False else 1/0

        try:
            if Pjnt==0 or sumEejnma==0:
                1/0
            else:
                output = (IA + 1/12 * Ojn)/((1-Pjnt)*1/12*sumEejnma)
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.18.'></a>

### 2.18. Función de cálculo del CD3-2    [Volver al inicio](#Indice)
<hr>
$\Large CD_\color{blue}{3-2,j,m,t}=CD_\color{blue}{3,j,m,t}\cdot \dfrac{{\sum\limits_{ma=m-3}^{m-14}Fe_\color{blue}{j,3-2,ma}}}{\bigg ( 1 - P_\color{blue}{j,2,t} \bigg ) \cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,2,ma}}}$
<hr>
$j$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large CD_\color{blue}{3-2,j,m,t}=CD_\color{blue}{3,j,m,t}\cdot \dfrac{{\sum\limits_{ma=m-3}^{m-14}Fe_\color{blue}{j,3-2,ma}}}{\bigg ( 1 - P_\color{blue}{j,2,t-1} \bigg ) \cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,2,ma}}}$

In [27]:
''' KPI Mensual '''
def CD3_2(m,t):
    name = 'CD3_2,j,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        n=2
        ''' CD3 KPI Mensual '''
        in_progress='$CD3'
        value = CDn(3,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CD3 = value if pd.isnull(value)==False else 1/0
        
        ''' P Variable Mensual'''
        in_progress='$P'
        if t>=2019:
            value = get_var_annualized('Pj,'+str(n)+',t',t)
            Pjnt = value if pd.isnull(value)==False else 1/0
        else:
            value = get_var('Pj,'+str(n)+',t',t,0)
            Pjnt = value if pd.isnull(value)==False else 1/0
        
        ''' Ee Variable Mensual '''
        in_progress='$sumEe'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('Eej,'+str(n)+',ma',i.year,i.month))
        value = sum(accum)
        sumEejnma = value if pd.isnull(value)==False else 1/0
        
        ''' Variable Mensual'''
        in_progress='$FE'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('FEj,3-2,ma',i.year,i.month))
        value = sum(accum)
        FE = value if pd.isnull(value)==False else 1/0

        try:
            if Pjnt==0 or sumEejnma==0:
                1/0
            else:
                output = CD3*FE/((1-Pjnt)*sumEejnma)
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.19.'></a>

### 2.19. Función de cálculo del CD2    [Volver al inicio](#Indice)
<hr>
$\Large CD_\color{blue}{2,j,m,t}=\dfrac{IA_\color{blue}{j,2,m,t}+\dfrac{1}{12}\cdot O_\color{blue}{j,2}}{\bigg (1-P_\color{blue}{j,2,t}\bigg )\cdot \dfrac{1}{12} \cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,2,ma}}}+CD_\color{blue}{3-2,j,m,t}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large CD_\color{blue}{2,j,m,t}=\dfrac{IA_\color{blue}{j,2,m,t}+\dfrac{1}{12}\cdot O_\color{blue}{j,2}}{\bigg (1-P_\color{blue}{j,2,t-1}\bigg )\cdot \dfrac{1}{12} \cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,2,ma}}}+CD_\color{blue}{3-2,j,m,t}$

In [28]:
''' KPI Mensual '''
def CD2(m,t):
    n=2
    name = 'CD'+str(n)+',j,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' IA KPI mensual '''
        in_progress='$IA'
        if n in [2,3]:
            value = IAjnmt(n,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            IA = value if pd.isnull(value)==False else 1/0
        elif n ==1:
            value = IAj1mt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            IA = value if pd.isnull(value)==False else 1/0
        
        ''' O Constante '''
        in_progress='$O'
        value = get_constant('Oj,'+str(n))
        Ojn = value if pd.isnull(value)==False else 1/0
        
        ''' P Variable Anual '''
        in_progress='$P'
        if t>=2019:
            value = get_var_annualized('Pj,'+str(n)+',t',t)
            Pjnt = value if pd.isnull(value)==False else 1/0
        else:
            value = get_var('Pj,'+str(n)+',t',t,0)
            Pjnt = value if pd.isnull(value)==False else 1/0

        ''' Ee Variable Mensual '''
        in_progress='$sumEe'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('Eej,'+str(n)+',ma',i.year,i.month))
        value = sum(accum)
        sumEejnma = value if pd.isnull(value)==False else 1/0
        
        ''' CD3_2 KPI Mensual'''
        in_progress='$CD3_2'
        value = CD3_2(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CD3_2_value = value if pd.isnull(value)==False else 1/0
        
        try:
            if Pjnt==0 or sumEejnma==0:
                1/0
            else:
                output = (IA + 1/12 * Ojn)/((1-Pjnt)*1/12*sumEejnma) + CD3_2_value
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.20.'></a>

### 2.20. Función de cálculo del Dtcs    [Volver al inicio](#Indice)
<hr>
$\Large Dtcs_\color{blue}{n,j,m,t}=\dfrac{\dfrac{INCC_\color{blue}{j,n,t}}{IPP_\color{blue}{FC}}-\dfrac{CONP_\color{blue}{j,n,t}}{IPP_\color{blue}{t-1}}}{\bigg ( 1-P_\color{blue}{j,n,t} \bigg)\cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,n,ma}}}\cdot IPP_\color{blue}{m-1}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large Dtcs_\color{blue}{n,j,m,t}=\dfrac{\dfrac{INCC_\color{blue}{j,n,t}}{IPP_\color{blue}{FC}}-\dfrac{CONP_\color{blue}{j,n,t}}{IPP_\color{blue}{t-1}}}{\bigg ( 1-P_\color{blue}{j,n,t-1} \bigg)\cdot {\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,n,ma}}}\cdot IPP_\color{blue}{m-1}$

In [29]:
''' KPI Mensual '''
def Dtcs(n,m,t):
    name = 'Dtcs'+str(n)+',j,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' INCC KPI Mensual '''
        in_progress='$INCC'
        value = INCCjnt(n,t+1)
        INCC = value if pd.isnull(value)==False else 1/0

        ''' CONP KPI Mensual '''
        in_progress='$CONP'
        value = CONPjnt(n,t+1)
        CONP = value if pd.isnull(value)==False else 1/0

        ''' IPP Variable Mensual '''
        in_progress='$IPP'
        value = datos_var_anual['INDICE_PRODUCTOR_ANIO_ANT'].loc[t]
        IPP = value if pd.isnull(value)==False else 1/0

        ''' IPPfc Variable Anual '''
        in_progress='$IPPfc'
        value = get_var_annualized('Ipp FC',t+1)
        IPPfc = value if pd.isnull(value)==False else 1/0

        ''' P Variable Anual '''
        in_progress='$Pjnt'
        '''Corregir'''
        if t>=2019:
            value = get_var_annualized('Pj,'+str(n)+',t',t)
            Pjnt = value if pd.isnull(value)==False else 1/0
        else:
            value = get_var('Pj,'+str(n)+',t',t,0)
            Pjnt = value if pd.isnull(value)==False else 1/0

        ''' Ee Variable Mensual '''
        in_progress='$sumEe'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('Eej,'+str(n)+',ma',i.year,i.month))
        value = sum(accum)
        sumEejnma = value if pd.isnull(value)==False else 1/0
        
        ''' IPP Variable mensual '''
        in_progress='$IPP'
        value = get_var('IPPm-1:',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        IPPm1 = value if pd.isnull(value)==False else 1/0
        
        try:
            if IPP==0 or IPPfc==0 or Pjnt==1 or sumEejnma==0:
                1/0
            else:
                output = (INCC/IPPfc - CONP/IPP)/((1-Pjnt)*sumEejnma)*IPPm1
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.21.'></a>

### 2.21. Función de cálculo del CDI    [Volver al inicio](#Indice)
<hr>
$\Large CDI_\color{blue}{1,j,m,t}=\dfrac{IA_\color{blue}{j,1,m,t}}{\bigg ( 1-P_\color{blue}{j,1,t}\bigg )\cdot \dfrac{1}{12}\cdot{\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,1,ma}}}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large CDI_\color{blue}{1,j,m,t}=\dfrac{IA_\color{blue}{j,1,m,t}}{\bigg ( 1-P_\color{blue}{j,1,t-1}\bigg )\cdot \dfrac{1}{12}\cdot{\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,1,ma}}}$

In [30]:
''' KPI Mensual '''
def CDI(m,t):
    n=1
    name = 'CDI'+str(n)+',j,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' IA KPI Mensual '''
        in_progress='$IA'
        if n in [2,3]:
            value = IAjnmt(n,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            IA = value if pd.isnull(value)==False else 1/0
        elif n ==1:
            value = IAj1mt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            IA = value if pd.isnull(value)==False else 1/0
        
        ''' P Variable Anual '''
        in_progress='$Pjnt'
        if t>=2019:
            value = get_var_annualized('Pj,'+str(n)+',t',t)
            Pjnt = value if pd.isnull(value)==False else 1/0
        else:
            value = get_var('Pj,'+str(n)+',t',t,0)
            Pjnt = value if pd.isnull(value)==False else 1/0
            
        ''' Ee Variable Mensual '''
        in_progress='$sumEe'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('Eej,'+str(n)+',ma',i.year,i.month))
        value = sum(accum)
        sumEejnma = value if pd.isnull(value)==False else 1/0
      
        try:
            if Pjnt==1 or sumEejnma==0:
                1/0
            else:
                output = IA/((1-Pjnt)*(1/12)*sumEejnma)
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.22.'></a>

### 2.22. Función de cálculo del CDA    [Volver al inicio](#Indice)
<hr>
$\Large CDA_\color{blue}{1,j,m,t}=\dfrac{IAAOM_\color{blue}{j,1,t}}{\bigg ( 1-P_\color{blue}{j,1,t}\bigg )\cdot{\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,1,ma}}}\cdot \dfrac{IPP_\color{blue}{m-1}}{IPP_\color{blue}{0}}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$n$ es el nivel de tensión, donde  $n \in \big\{ 1,2,3,4\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

<hr>
<b>Modificado por: </b>
$\Large CDA_\color{blue}{1,j,m,t}=\dfrac{IAAOM_\color{blue}{j,1,t}}{\bigg ( 1-P_\color{blue}{j,1,t-1}\bigg )\cdot{\sum\limits_{ma=m-3}^{m-14}Ee_\color{blue}{j,1,ma}}}\cdot \dfrac{IPP_\color{blue}{m-1}}{IPP_\color{blue}{0}}$

In [31]:
''' KPI Mensual '''
def CDA(m,t):
    n=1
    name = 'CDA'+str(n)+',j,'+str(m)+','+str(t)
    t=t-1
    try:
        ''' IAAOM KPI Anual '''
        in_progress='$IAAOM'
        value = IAAOMjnt(n,t+1)
        IAAOM = value if pd.isnull(value)==False else 1/0
        
        ''' P Variable Mensual '''
        in_progress='$Pjnt'
        value = get_var_annualized('Pj,'+str(n)+',t',t)
        Pjnt = value if pd.isnull(value)==False else 1/0
        
        ''' IPP Variable mensual '''
        in_progress='$IPP'
        value = get_var('IPPm-1:',month_correction(t+1, m-1)[0],month_correction(t+1, m-1)[1])
        IPPm1 = value if pd.isnull(value)==False else 1/0
        
        ''' IPP0 Constante '''
        in_progress='$IPP0'
        value = get_constant('IPPo:')
        IPP0 = value if pd.isnull(value)==False else 1/0
        
        ''' Ee Variable Mensual '''
        in_progress='$sumEe'
        dates = dates_to_iter(month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        accum = []
        for i in dates:
            accum.append(get_var('Eej,'+str(n)+',ma',i.year,i.month))
        value = sum(accum)
        sumEejnma = value if pd.isnull(value)==False else 1/0

        try:
            if Pjnt==1 or sumEejnma==0 or IPP0==0:
                1/0
            else:
                output = (IAAOM/((1-Pjnt)*sumEejnma)*IPPm1/IPP0)
                log(name,0)
                save_to_local_file(name,t,m,output,2)
                return output
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.23.'></a>

### 2.23. Función de cálculo del Dt4    [Volver al inicio](#Indice)
<hr>
$\Large Dt_\color{blue}{4,R,m,t}=\dfrac{CD_\color{blue}{4,R,m,t}}{1-PR_\color{blue}{4,R,m,t}}$
<hr>
$R$ no afecta en el cálculo

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [32]:
''' KPI Mensual '''
def Dt4(m,t):
    name = 'Dt4,R,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' CD4 KPI Mensual '''
        in_progress='$CD4'
        value = CD4Rmt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CD4 = value if pd.isnull(value)==False else 1/0
        
        ''' PR4 KPI Mensual '''
        in_progress='$PR4'
        value = get_var('PR 4,R,m,t=P 4,R,m,t',month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        PR4 = value if pd.isnull(value)==False else 1/0

        try:
            if PR4==1:
                1/0
            else:
                output = CD4/(1 - PR4)
                log(name,0)
                save_to_local_file(name,t,m,output,1)
                return 1
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.24.'></a>

### 2.24. Función de cálculo del Dt3 y Dt2    [Volver al inicio](#Indice)
<hr>
$\Large Dt_\color{blue}{n,R,m,t}=\dfrac{CD_\color{blue}{4,R,m,t}}{1-PR_\color{blue}{n,j,m,t}}+CD_\color{blue}{n,j,m,t}+Dtcs_\color{blue}{n,j,m,t}$
<hr>
$j$ no afecta en el cálculo 

$R$ no afecta en el cálculo

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

In [33]:
''' KPI Mensual '''
def Dtn(n,m,t):
    name = 'Dt'+str(n)+',R,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' KPI Mensual '''
        in_progress='$CD4'
        value = CD4Rmt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CD4 = value if pd.isnull(value)==False else 1/0
        
        ''' PR Variable Mensual '''
        in_progress='$PR'
        value = get_var('PR'+str(n)+',j,m,t',t+1,m)
        PR = value if pd.isnull(value)==False else 1/0

        ''' CD KPI Mensual '''
        if n==3:
            in_progress='$CD3'
            value = CDn(3,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            CD = value if pd.isnull(value)==False else 1/0
        else:
            in_progress='$CD2'
            value = CD2(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
            CD = value if pd.isnull(value)==False else 1/0
        
        ''' Dtcs KPI Mensual '''
        in_progress='$Dtcs'
        value = Dtcs(n,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        Dtcs_value = value if pd.isnull(value)==False else 1/0
        try:
            if PR==1:
                1/0
            else:
                output = CD4/(1 - PR) + CD + Dtcs_value
                log(name,0)
                save_to_local_file(name,t,m,output,1)
                return 1
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='2.25.'></a>

### 2.25. Función de cálculo del Dt1    [Volver al inicio](#Indice)
<hr>
$\Large Dt_\color{blue}{1,R,m,t}=\dfrac{CD_\color{blue}{4,R,m,t}}{1-PR_\color{blue}{1,j,m,t}}+\dfrac{CD_\color{blue}{2,j,m,t}}{1-P_\color{blue}{j,1,t}}+CDI_\color{blue}{1,j,m,t}+CDA_\color{blue}{1,j,m,t}+Dtcs_\color{blue}{1,j,m,t}$
<hr>
$j$ no afecta en el cálculo

$R$ no afecta en el cálculo

$m$ es el mes, donde $m \in \big\{ 1,2,3,4,5,6,7,8,9,10,11,12\big\}$

$t$ es el año, donde $t >= 2019$ debido a que el año inicial con datos es 2018

<hr>
<b>Modificado por: </b>
$\Large Dt_\color{blue}{1,R,m,t}=\dfrac{CD_\color{blue}{4,R,m,t}}{1-PR_\color{blue}{1,j,m,t}}+\dfrac{CD_\color{blue}{2,j,m,t}}{1-P_\color{blue}{j,1,t-1}}+CDI_\color{blue}{1,j,m,t}+CDA_\color{blue}{1,j,m,t}+Dtcs_\color{blue}{1,j,m,t}$

In [34]:
''' KPI Mensual '''
def Dt1(m,t):
    name = 'Dt1,R,'+str(m)+','+str(t)
    in_progress=''
    t=t-1
    try:
        ''' CD4 KPI Mensual '''
        in_progress='$CD4'
        value = CD4Rmt(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CD4 = value if pd.isnull(value)==False else 1/0
        
        ''' PR1 KPI Mensual '''
        in_progress='$PR1'
        
        value = get_var('PR1,j,m,t',month_correction(t+1, m)[0],month_correction(t+1, m)[1])
        PR1 = value if pd.isnull(value)==False else 1/0
        
        ''' Variable Anual '''
        in_progress='$P1'
        value = get_var_annualized('Pj,1,t',t)
        P1 = value if pd.isnull(value)==False else 1/0
        
        ''' KPI Mensual '''
        in_progress='$CDI1'
        value = CDI(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CDI1 = value if pd.isnull(value)==False else 1/0
        
        ''' KPI Mensual '''
        in_progress='$CDA1'
        value = CDA(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CDA1 = value if pd.isnull(value)==False else 1/0
        
        ''' KPI Mensual '''
        in_progress='$CD2'
        value = CD2(month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        CD2_value = value if pd.isnull(value)==False else 1/0
        
        ''' KPI Mensual '''
        in_progress='$Dtcs1'
        value = Dtcs(1,month_correction(t+1, m)[1],month_correction(t+1, m)[0])
        Dtcs1 = value if pd.isnull(value)==False else 1/0
  
        try:
            if PR1==1 or P1==1:
                1/0
            else:
                output = CD4/(1 - PR1) + CD2_value/(1-P1) + CDI1 + CDA1 + Dtcs1
                log(name,0)
                save_to_local_file(name,t,m,output,1)
                return 1
        except:
            log(name,2)
    except:
        log(name+in_progress,1)

<a id='3.'></a>

## 3. Cálculo de los KPI   [Volver al inicio](#Indice)

<a id='3.1.'></a>

### 3.1. Cálculo para fecha actual   [Volver al inicio](#Indice)

In [35]:
mes = int(timestamp[5:7])
#mes = 1
anio = int(timestamp[:4])
#anio = 2020
try:
    ''' Ejecuta el cálculo Dt4 '''
    r4 = Dt4(mes,anio)
    if r4==1:
        print('-----------Dt4 se ha calculado con éxito----\n')
    else:
        print('-----------Dt4 no se pudo calcular----------\n')

    ''' Ejecuta el cálculo Dt3 '''
    r3 = Dtn(3,mes,anio)
    if r3==1:
        print('-----------Dt3 se ha calculado con éxito----\n')
    else:
        print('-----------Dt3 no se pudo calcular----------\n')

    ''' Ejecuta el cálculo Dt2 '''
    r2 = Dtn(2,mes,anio)
    if r2==1:
        print('-----------Dt2 se ha calculado con éxito----\n')
    else:
        print('-----------Dt2 no se pudo calcular----------\n')

    ''' Ejecuta el cálculo Dt1 '''
    r1 = Dt1(mes,anio)
    if r1==1:
        print('-----------Dt1 se ha calculado con éxito----\n')
    else:
        print('-----------Dt1 no se pudo calcular----------\n')
    tr_niveles['ANIO']=anio
    tr_niveles['PERIODO']=mes
    tr_niveles = tr_niveles.drop_duplicates()
    if (r1+r2+r3+r4)==4:
        tr_niveles.to_csv('tmp/tr_niveles_'+str(timestamp_id)+'.txt', index=None,sep='$')
        log_file.to_csv('tmp/tr_log_calculo_'+str(timestamp_id)+'.txt', index=None,sep='$')
        datos_anualizados.to_csv('tmp/tr_datos_anualizados_'+str(timestamp_id)+'.txt', index=None,sep='$')
        move_local_to_hdfs('tmp/tr_niveles_'+str(timestamp_id)+'.txt', '/user/perdidas_co/proy_davinci/tr_niveles/')
        move_local_to_hdfs('tmp/tr_log_calculo_'+str(timestamp_id)+'.txt', '/user/perdidas_co/proy_davinci/tr_log_calculo/')
        move_local_to_hdfs('tmp/tr_datos_anualizados_'+str(timestamp_id)+'.txt', '/user/perdidas_co/proy_davinci/tr_datos_anualizados/')
except:
    log_file.to_csv('tmp/tr_log_calculo_'+str(timestamp_id)+'.txt', index=None,sep='$')
    move_local_to_hdfs('tmp/tr_log_calculo_'+str(timestamp_id)+'.txt', '/user/perdidas_co/proy_davinci/tr_log_calculo/')
    print('---------------------------------------------------------------')
    print('No se han actualizado los datos debido a que falta información.')
    print('---------------------------------------------------------------')
Download = '''<a style=" background-color: green;
  border: none;
  color: white;
  padding: 10px 15px;
  text-align: center;
  text-decoration: none;
  height: 40px;
  width: 170px;
  display: inline-block;
  font_weight: bold" href="tmp/tr_log_calculo_%s.txt" download ><b>Descarga Log</a>''' % timestamp_id
HTML(Download)

Info: KPI RCj,4,2019: - calculado correctamente.
Info: KPI BRAEj,4,2019: - calculado correctamente.
Info: KPI RCj,4,2019: - calculado correctamente.
Info: KPI BRAEj,4,2019: - calculado correctamente.
Info: KPI BRANEj,4,2019 - calculado correctamente.
Info: KPI BRAj,4,2019 - calculado correctamente.
Info: KPI RCj,4,2019: - calculado correctamente.
Info: KPI IAAj,4,2019 - calculado correctamente.
Info: KPI IAAOMj,4,2019 - calculado correctamente.
Info: KPI IRMj,4,2019: - calculado correctamente.
Info: KPI IMj,4R,7,2019: - calculado correctamente.
Info: KPI CD4,R,7,2019 - calculado correctamente.
Info: KPI Dt4,R,7,2019 - calculado correctamente.
-----------Dt4 se ha calculado con éxito----

Info: KPI RCj,4,2019: - calculado correctamente.
Info: KPI BRAEj,4,2019: - calculado correctamente.
Info: KPI RCj,4,2019: - calculado correctamente.
Info: KPI BRAEj,4,2019: - calculado correctamente.
Info: KPI BRANEj,4,2019 - calculado correctamente.
Info: KPI BRAj,4,2019 - calculado correctamente.
Inf

In [36]:
tr_niveles

,INDICADOR,ANIO,PERIODO,VALOR,NIVEL,TIMESTAMP
0,"RCj,4,2019:",2019,7,6.012160e+10,8,2019-07-05 13:43:27
0,"BRAEj,4,2019:",2019,7,1.625280e+12,7,2019-07-05 13:43:27
0,"BRANEj,4,2019",2019,7,3.250559e+10,6,2019-07-05 13:43:27
0,"BRAj,4,2019",2019,7,1.657785e+12,5,2019-07-05 13:43:27
0,"IAAj,4,2019",2019,7,2.586576e+11,4,2019-07-05 13:43:27
0,"IAAOMj,4,2019",2019,7,4.214481e+10,4,2019-07-05 13:43:27
0,"IRMj,4,2019:",2019,7,0.000000e+00,4,2019-07-05 13:43:27
0,"IMj,4R,7,2019:",2019,7,2.510126e+10,3,2019-07-05 13:43:27
0,"CD4,R,7,2019",2019,7,1.994431e+01,2,2019-07-05 13:43:27
0,"Dt4,R,7,2019",2019,7,2.013966e+01,1,2019-07-05 13:43:27
